In [1]:
import mand.core

from mand.core import Entity, node, Context
from mand.core import PrintMonitor, SummaryMonitor

In [2]:
class O(Entity):

    @node
    def X(self):
        return 10
    
    @node
    def Y(self):
        return self.X()**3
    
    @node
    def eps(self):
        return 1e-4
    
    @node 
    def delta(self):
        # bad: single-sided delta
        eps = self.eps()
        y = self.Y()
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump}, name='X-up')
        with c:
            y_bump = self.Y()
        return (y_bump-y)/eps
    
    @node 
    def gamma(self):
        eps = self.eps()
        d = self.delta()
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump})
        with c:
            d_bump = self.delta()
        return (d_bump-d)/eps

In [3]:
with PrintMonitor():
    o = O()
    d = o.delta()
    print 'delta:', d

 Context set value: <mand.noval.NoVal..., ctx: Root, key: O@113a1c3d0/O:delta
 GetValue begin ctx: Root, key: O@113a1c3d0/O:delta
   GetValue/Calc begin ctx: Root, key: O@113a1c3d0/O:delta
     Context set value: <mand.noval.NoVal..., ctx: Root, key: O@113a1c3d0/O:eps
     GetValue begin ctx: Root, key: O@113a1c3d0/O:eps
       GetValue/Calc begin ctx: Root, key: O@113a1c3d0/O:eps
         GetValue/Calc end ctx: Root, key: O@113a1c3d0/O:eps
       GetValue end value: 0.0001, ctx: Root, key: O@113a1c3d0/O:eps
     Context set value: <mand.noval.NoVal..., ctx: Root, key: O@113a1c3d0/O:Y
     GetValue begin ctx: Root, key: O@113a1c3d0/O:Y
       GetValue/Calc begin ctx: Root, key: O@113a1c3d0/O:Y
         Context set value: <mand.noval.NoVal..., ctx: Root, key: O@113a1c3d0/O:X
         GetValue begin ctx: Root, key: O@113a1c3d0/O:X
           GetValue/Calc begin ctx: Root, key: O@113a1c3d0/O:X
             GetValue/Calc end ctx: Root, key: O@113a1c3d0/O:X
           GetValue end value: 10

In [4]:
d = o.delta()
print 'delta:', d
g = o.gamma()
print 'gamma:', g

delta: 300.003000009
gamma: 60.0006046625


In [5]:
c = Context.current()
cbm = o.gamma
n = c.getCBM(cbm)
n.printInputGraph()

 <O@113a1c3d0/O:gamma in Root>
   <O@113a1c3d0/O:eps in Root>
   <O@113a1c3d0/O:delta in Root:4547403672>
     <O@113a1c3d0/O:eps in Root:4547403672>
     <O@113a1c3d0/O:Y in Root:4547403672>
       <O@113a1c3d0/O:X in Root:4547403672>
     <O@113a1c3d0/O:Y in Root:4547403672:X-up>
       <O@113a1c3d0/O:X in Root:4547403672:X-up>
     <O@113a1c3d0/O:X in Root:4547403672>
   <O@113a1c3d0/O:delta in Root>
     <O@113a1c3d0/O:eps in Root>
     <O@113a1c3d0/O:Y in Root:X-up>
       <O@113a1c3d0/O:X in Root:X-up>
     <O@113a1c3d0/O:Y in Root>
       <O@113a1c3d0/O:X in Root>
     <O@113a1c3d0/O:X in Root>
   <O@113a1c3d0/O:X in Root>


In [6]:
class O2(Entity):

    @node
    def X(self):
        return 10
    
    @node
    def Y(self):
        return self.X()**3
    
    @node
    def eps(self):
        return 1e-4
    
    @node
    def upCtx(self):
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump}, name='X-up')
        return c
    
    @node
    def downCtx(self):
        x_bump = self.X() - self.eps()
        c = Context({self.X: x_bump}, name='X-down')
        return c
    
    @node 
    def delta(self):
        eps = self.eps()
        with self.upCtx():
            y_up = self.Y()
        with self.downCtx():
            y_down = self.Y()
        return (y_up-y_down)/eps/2
    
    @node 
    def gamma(self):
        eps = self.eps()
        with self.upCtx():
            delta_up = self.delta()
        with self.downCtx():
            delta_down = self.delta()
        return (delta_up-delta_down)/eps/2
    
with PrintMonitor():
    o2 = O2()
    d = o2.delta()
    print 'delta:', d
    g = o2.gamma()
    print 'gamma:', g

 Context set value: <mand.noval.NoVal..., ctx: Root, key: O2@113a310d0/O2:delta
 GetValue begin ctx: Root, key: O2@113a310d0/O2:delta
   GetValue/Calc begin ctx: Root, key: O2@113a310d0/O2:delta
     Context set value: <mand.noval.NoVal..., ctx: Root, key: O2@113a310d0/O2:eps
     GetValue begin ctx: Root, key: O2@113a310d0/O2:eps
       GetValue/Calc begin ctx: Root, key: O2@113a310d0/O2:eps
         GetValue/Calc end ctx: Root, key: O2@113a310d0/O2:eps
       GetValue end value: 0.0001, ctx: Root, key: O2@113a310d0/O2:eps
     Context set value: <mand.noval.NoVal..., ctx: Root, key: O2@113a310d0/O2:upCtx
     GetValue begin ctx: Root, key: O2@113a310d0/O2:upCtx
       GetValue/Calc begin ctx: Root, key: O2@113a310d0/O2:upCtx
         Context set value: <mand.noval.NoVal..., ctx: Root, key: O2@113a310d0/O2:X
         GetValue begin ctx: Root, key: O2@113a310d0/O2:X
           GetValue/Calc begin ctx: Root, key: O2@113a310d0/O2:X
             GetValue/Calc end ctx: Root, key: O2@113a31

In [7]:
with SummaryMonitor():
    o2a = O2()
    d = o2a.delta()
    print 'delta:', d
    g = o2a.gamma()
    print 'gamma:', g

delta: 300.000000009
gamma: 59.9999992801
Compute activity:
               Context:     6
              GetValue:    40
         GetValue/Calc:    20
